In [1]:
import requests
import json
from datetime import date, datetime, time

In [2]:
!pip freeze | grep -i "^lang"

langchain==0.2.16
langchain-community==0.2.16
langchain-core==0.2.43
langchain-google-community==1.0.8
langchain-google-vertexai==1.0.6
langchain-ollama==0.1.3
langchain-text-splitters==0.2.4
langgraph==0.2.38
langgraph-checkpoint==2.0.8
langgraph-sdk==0.1.43
langsmith==0.1.144


## Tools

### Get trains

In [3]:
def get_connections(
    from_location: str, 
    to_location: str, 
    date: date=None, 
    time: time=None):
    
    r = requests.get(
        "http://transport.opendata.ch/v1/connections", 
        params={
            'from': from_location, 
            'to': to_location, 
            'date': date, 
            'time': time}
    )
    
    if r.raise_for_status():
        return "No connection found"
    
    return [
        {
            'departure': datetime.fromtimestamp(x['from']['departureTimestamp']).time().strftime(format="%H:%M"),
            'arrival': datetime.fromtimestamp(x['to']['arrivalTimestamp']).time().strftime(format="%H:%M")
        } for x in r.json().get('connections')
    ]

In [4]:
get_connections('Lausanne', 'Lugano')

[{'departure': '19:40', 'arrival': '23:58'},
 {'departure': '20:34', 'arrival': '01:25'},
 {'departure': '23:44', 'arrival': '07:58'},
 {'departure': '00:04', 'arrival': '08:58'}]

### Get data from Tripadvisor

In [5]:
def _query_tripadvisor_api(location: str, location_type: str):
    url = 'https://www.tripadvisor.com/data/graphql/ids'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36', 
        'Content-Type': 'application/json'
    }
    body = json.dumps([
        {"variables":
         {"request":
          {"filters":
           {"dataTypes":["LOCATION"],"locationTypes":[location_type]},
           "locale":"en-US",
           "query":location,
           "offset":0,
           "scope":{
               "locationId":1,"center":None
           },
           "locationIdsToExclude":[],
           "categoryFilterIds":["DESTINATIONS","RESTAURANTS","ATTRACTIONS","HOTELS","ACTIVITIES","VACATION_RENTALS"],
           "additionalFields":["SNIPPET","MENTION_COUNT"],
           "limit":30
          }
         },
         "extensions":
         {"preRegisteredQueryId":"d65d51b7e2ed4f40"}
        }
    ])
    r = requests.post(url, data=body, headers=headers)
    if r.raise_for_status():
        return "Error"

    try:
        return [
            {
                'name': x['details']['localizedName'],
                'link': "https://www.tripadvisor.com" + x['details']['defaultUrl'],
                'description': x['details']['locationDescription'],
                'rating': x['details']['reviewSummary']['rating'],
                'reviews': x['details']['reviewSummary']['count']
                
            } for x in r.json()[0]['data']['SERP_getSearchResultsList']['clusters'][0]['sections'][1]['results']]
    except: 
        return r.json()

### Get hotels from Tripadvisor

In [6]:
def search_hotels(location: str):
    return _query_tripadvisor_api(location, location_type='ACCOMMODATION')

In [7]:
r = search_hotels('Niort')

In [8]:
r

[{'name': 'Mercure Niort Marais Poitevin',
  'link': 'https://www.tripadvisor.com/Hotel_Review-g196667-d197093-Reviews-Mercure_Niort_Marais_Poitevin-Niort_Deux_Sevres_Nouvelle_Aquitaine.html',
  'description': 'The Mercure Niort Marais Poitevin, in the centre of town and close to the historic quarter, welcomes you in the shade of large trees in its garden. Offers quiet, spacious and comfortable rooms with their own bathroom, WC, flatscreen TV with international channels, broadband Internet and minibar. Our La Veranda du Dauzac restaurant with its refined, contemporary decor, will enlighten you with an inventive and flavoursome cuisine in tune with the seasons. Shady terrace, pool and secure private parking.',
  'rating': 4,
  'reviews': 712},
 {'name': 'Grand Hotel Niort Centre',
  'link': 'https://www.tripadvisor.com/Hotel_Review-g196667-d481699-Reviews-Grand_Hotel_Niort_Centre-Niort_Deux_Sevres_Nouvelle_Aquitaine.html',
  'description': 'Welcome to Grand Hotel Niort Centre, a nice op

### Get restaurants from Tripadvisor

In [9]:
def search_restaurants(location: str):
    return _query_tripadvisor_api(location, location_type='EATERY')

In [10]:
r = search_restaurants('Niort')

In [11]:
r

[{'name': 'Restaurant du Donjon',
  'link': 'https://www.tripadvisor.com/Restaurant_Review-g196667-d1212588-Reviews-Restaurant_du_Donjon-Niort_Deux_Sevres_Nouvelle_Aquitaine.html',
  'description': None,
  'rating': 4.5,
  'reviews': 856},
 {'name': 'La Villa',
  'link': 'https://www.tripadvisor.com/Restaurant_Review-g196667-d4072542-Reviews-La_Villa-Niort_Deux_Sevres_Nouvelle_Aquitaine.html',
  'description': None,
  'rating': 4,
  'reviews': 949},
 {'name': 'Plaisirs des Sens',
  'link': 'https://www.tripadvisor.com/Restaurant_Review-g196667-d1212666-Reviews-Plaisirs_des_Sens-Niort_Deux_Sevres_Nouvelle_Aquitaine.html',
  'description': 'Welcome to the Plaisirs des Sens. We propose a fine and creative french cuisine. Honoured by many Quality Labels, our menus are all home-made, prepared with only fresh local products. We also have a special focus on wines from all France regions. In the center of Niort (Public Parking Place de la Brèche), come and discover our cosy atmosphere and our 

In [12]:
from langchain_core.tools import tool

In [13]:
@tool
def train_assistant(
    from_location: str, 
    to_location: str, 
    date: date=None, 
    time: time=None):
    """
    Search for train connections between cities, on a given date (default today) and time.
    """
    
    r = requests.get(
        "http://transport.opendata.ch/v1/connections", 
        params={
            'from': from_location, 
            'to': to_location, 
            'date': date, 
            'time': time}
    )
    
    if r.raise_for_status():
        return "No connection found"
    
    return [
        {
            'departure': datetime.fromtimestamp(x['from']['departureTimestamp']).time().strftime(format="%H:%M"),
            'arrival': datetime.fromtimestamp(x['to']['arrivalTimestamp']).time().strftime(format="%H:%M")
        } for x in r.json().get('connections')
    ]


@tool
def search_hotels_assistant(location: str):
    """
    Search for hotels for a given location
    """
    return _query_tripadvisor_api(location, location_type='ACCOMMODATION')


@tool
def search_restaurants_assistant(location: str):
    """
    Search for restaurants for a given location
    """
    return _query_tripadvisor_api(location, location_type='EATERY')

## Assistant

### State

Explain concepts about messages and states

In [14]:
from typing import Annotated, Dict
from typing_extensions import TypedDict

from langgraph.graph.message import AnyMessage, add_messages
from langchain_core.messages import HumanMessage, AIMessage

In [15]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

### Agent

Explain concept of runnable and how the agent works

In [16]:
from langchain_core.runnables import Runnable

In [17]:
class Agent:
    def __init__(self, runnable: Runnable):
        self.runnable = runnable

    def __call__(self, state: Dict):
        result = self.runnable.invoke(state)
        return {'messages': result}

### LLM 

In [18]:
from langchain_google_vertexai import ChatVertexAI

In [19]:
llm = ChatVertexAI(model="gemini-1.5-flash", temperature=0).bind_tools([search_hotels_assistant, search_restaurants_assistant, train_assistant])

In [20]:
from langchain_core.prompts import ChatPromptTemplate

In [21]:
assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful customer support assistant for a travel agency "
            " Use the provided tools to look for train schedules, hotels and resturants"
        ),
        ("placeholder", "{messages}"),
    ]
)

In [22]:
assistant_runnable = assistant_prompt | llm

In [23]:
assistant = Agent(assistant_runnable)

In [24]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import tools_condition, ToolNode


builder = StateGraph(State)


# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode([search_hotels_assistant, search_restaurants_assistant, train_assistant]))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
)
builder.add_edge("tools", "assistant")

# The checkpointer lets the graph persist its state
# this is a complete memory for the entire graph.
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [31]:
import time

config = {"configurable": {"thread_id": 42}}
questions = [
    'When is the next train to Luzern ?',
    'Lausanne',
    'Can you show me the top 5 best hotel based on the rating ?',
    'Show me the top 3 restaurants'
]

for query in questions:
    print(f"[USER] {query}")
    inputs = {"messages": [HumanMessage(content=query)]}
    print("[ASSISTANT]: I am thinking...")
    events = graph.stream(
        {"messages": ("user", query)}, 
        config, 
        stream_mode="values",
        #output_keys=['agent']
    )
    for output in events:
    #     output['messages'][-1].pretty_print()
        pass
    print(f"[ASSISTANT]: {output['messages'][-1].content.strip()}")
    time.sleep(3)

[USER] When is the next train to Luzern ?
[ASSISTANT]: I am thinking...
[ASSISTANT]: From where?
[USER] Lausanne
[ASSISTANT]: I am thinking...
[ASSISTANT]: The next trains to Luzern from Lausanne leave at 19:40, 20:17, 20:40 and 21:17.
[USER] Can you show me the top 5 best hotel based on the rating ?
[ASSISTANT]: I am thinking...
[ASSISTANT]: Here are the top 5 hotels in Luzern based on rating:

1. Hotel Schweizerhof Luzern (Rating: 5, Reviews: 1229)
2. Art Deco Hotel Montana (Rating: 4.5, Reviews: 3081)
3. Hotel des Balances (Rating: 4.5, Reviews: 2581)
4. HERMITAGE Lake Lucerne - Beach Club & Lifestyle Hotel (Rating: 4.5, Reviews: 1560)
5. Hotel Central Luzern (Rating: 4.5, Reviews: 568)
[USER] Show me the top 3 restaurants
[ASSISTANT]: I am thinking...
[ASSISTANT]: Here are the top 3 restaurants in Luzern based on rating:

1. Bolero Restaurante (Rating: 4.5, Reviews: 2196)
2. Restaurant La Cucina (Rating: 4.5, Reviews: 1870)
3. Scala Restaurant - Art Deco Hotel Montana (Rating: 4.5,